First try of getting an Amplitude vs Time Graph.

While this does achieve a graph, I was mainly just trying to understand the terms and functions of how to deal with the graph, hence heavy the commenting.

In [ ]:
###import pathlib #find directory of a folder so that it works for other OSs
###import os #read the contents within folder
###------------### Only necessary when we do the searching for melody directory.


import pygame #start and stop of track playing
import scipy #has the fft <-- see OneNote page for explanation
#https://realpython.com/python-scipy-fft/
#http://www.differencebetween.net/technology/difference-between-fft-and-dft/

from scipy.io import wavfile


audio_to_identify = "C:\$$$AUDIO FILES\C3.wav"
rate, data = wavfile.read(audio_to_identify)
#mmap = False <-- file is not too large that it has to be mapped
# like any file, we have to read the information inside of it to do stuff with it.


#print(rate) #define rate: https://www.adobe.com/uk/creativecloud/video/discover/audio-sampling.html
#print(data) #define data: https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html
#data == the shape of the wave

##GETTING TO KNOW THE FUNCTIONS OF SCIPY TO DO WITH WAV
print(f"number of channels = {data.shape[1]}")
length = data.shape[0] / rate #whole entire audio file/ the frequency of wave
print(f"length = {length}s")
print(data.shape[0])

##PLOTTING THE WAVE!!! -- Amplitude vs Time
import matplotlib.pyplot as plt
import numpy as np
time = np.linspace(0., length, data.shape[0])
plt.plot(time, data[:, 0], label="Left channel")
plt.plot(time, data[:, 1], label="Right channel") #which speaker sound coming out of
plt.legend() #where the key for the graph is
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()

NORMALISATION -- important so sound can actually be heard. If amplitude/ power (power is the official term in programming for sound, but I use power and amplitude interchangeably as it means the same thing) is too small, then the tone will be harder to distinguish.

In [ ]:
##NORMALISATION

import matplotlib.pyplot as plt
from scipy.io import wavfile
import numpy as np

in_wav = "C:\$$$AUDIO FILES\$$FULL_c3_major_scale.wav"
rate, data = wavfile.read(in_wav) 
##rate = data points per second
normalised_tone = np.int16((data[:,0]/data[:,0].max()) * 32767)
# this is the tone definition normalizes to 32k bits = standard height for the tone to be identified
#32767 = 2^15 - 1, thus putting the .WAV file into the standard 16-bit height
#[:,0] is to get the data for one channel (speaker) because both channels will have similar data
#so it does not make sense to have data from both channels.
plt.plot(normalised_tone)
plt.xlabel("Point in Time")
plt.ylabel("Amplitude")
plt.ylim([-32767, 32767])
plt.show()
#Normalized tone will be used for further analysis

CREATING LIST OF ALL NOTES ON PIANO

Realistically, especially because the aim is to take human pitch and turn that into MIDI, the possible notes a human can reach are all going to be on a piano, and even then this is a very generous range.

In [ ]:
with open("C:\$$$AUDIO FILES\TO_USE-Note_To_MIDI_Table.csv") as f:
    formatted_all_notes = []
    all_notes = [line.split() for line in f]
    for lst in all_notes[1:]:
        for elem in lst:
            elem = elem.split(",")
            for i in range (1,4):
                if i == 1 or i == 3:
                    elem[i] = int(elem[i])
                else:
                    elem[i] = float(elem[i])
            formatted_all_notes.append(elem)
print(formatted_all_notes)

FIRST FORAY INTO RFFT

This is just a short note for why rfft is used in this code. For explanation of what it does (i.e. what is Fourier Transformation), please see OneNote.

rfft is used because:
(a) fft itself is quicker than dft
(b) rfft only takes values above zero -- sound is a sine curve so both positive and negative values are included
but in this case, not necessary because it is just a mirror image so dealing with positive numbers is ok.

In [ ]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import numpy as np

from scipy.fft import rfft, rfftfreq #new!!!

in_wav = "C:\$$$AUDIO FILES\C3.wav"
rate, data = wavfile.read(in_wav) 
normalised_tone = np.int16((data[:,0]/data[:,0].max()) * 32767)
plt.plot(normalised_tone)
plt.xlabel("Point in Time")
plt.ylabel("Amplitude")
plt.ylim([-32767, 32767])
plt.show()


# ------------------------------ #
yf = rfft(normalised_tone) #calculates the transform itself
xf = rfftfreq(len(normalised_tone), 1 / rate) #calculates the frequencies in centre of each bin in output of fft

plt.plot(xf, np.abs(yf))
plt.show()

READING ONE TONE FROM WHOLE FILE -- PIANO

In [35]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import numpy as np

from scipy.fft import rfft, rfftfreq

in_wav = "C:\$$$AUDIO FILES\VOICE_C3.wav"

### STEP ONE: GET RID OF ONE CHANNEL IF ORIGINAL RECORDING HAS TWO SO EASIER TO DEAL WITH + NORMALISE FILE 
rate, data = wavfile.read(in_wav)
normalised_tone = np.int16((data[:,0]/data[:,0].max()) * 32767)

### STEP 2: HOW TO GET TONE OF WHOLE FILE -- ONLY FOR ONE PITCH FILE

def single_pitch_to_MIDI(normalised_tone,rate):
    yf = rfft(normalised_tone) 
    xf = rfftfreq(len(normalised_tone),1/rate) 

    points_per_freq = len(xf) / (rate/2)
    # This is the number of points plotted per 1Hz of frequency.
    # The maximum frequency possibly reached is half the sample rate
    to_cut_off = int(points_per_freq * 4200) # Sets up point to cut sound off at 4200Hz because piano's highest key is 4186Hz

    yf = yf[:to_cut_off+1] # Cut off all points above 4200 Hz
    xf = xf[:to_cut_off+1]



    A = 0
    Amax = 0
    possible_tones = [['A0', 21, 27.5, 0], ['As0/Bb0', 22, 29.14, 0], ['B0', 23, 30.87, 0], ['C1', 24, 32.7, 0], ['Cs1/Db1', 25, 34.65, 0], ['D1', 26, 36.71, 0], ['Ds1/Eb1', 27, 38.89, 0], ['E1', 28, 41.2, 0], ['F1', 29, 43.65, 0], ['Fs1/Gb1', 30, 46.25, 0], ['G1', 31, 49.0, 0], ['Gs1/Ab1', 32, 51.91, 0], ['A1', 33, 55.0, 0], ['As1/Bb1', 34, 58.27, 0], ['B1', 35, 61.74, 0], ['C2', 36, 65.41, 0], ['Cs1/Db2', 37, 69.3, 0], ['D2', 38, 73.42, 0], ['Ds1/Eb2', 39, 77.78, 0], ['E2', 40, 82.41, 0], ['F2', 41, 87.31, 0], ['Fs1/Gb2', 42, 92.5, 0], ['G2', 43, 98.0, 0], ['Gs1/Ab2', 44, 103.83, 0], ['A2', 45, 110.0, 0], ['As1/Bb2', 46, 116.54, 0], ['B2', 47, 123.47, 0], ['C3', 48, 130.81, 0], ['Cs1/Db3', 49, 138.59, 0], ['D3', 50, 146.83, 0], ['Ds1/Eb3', 51, 155.56, 0], ['E3', 52, 164.81, 0], ['F3', 53, 174.61, 0], ['Fs1/Gb3', 54, 185.0, 0], ['G3', 55, 196.0, 0], ['Gs1/Ab3', 56, 207.65, 0], ['A3', 57, 220.0, 0], ['As1/Bb3', 58, 233.08, 0], ['B3', 59, 246.94, 0], ['C4', 60, 261.63, 0], ['Cs1/Db4', 61, 277.18, 0], ['D4', 62, 293.66, 0], ['Ds1/Eb4', 63, 311.13, 0], ['E4', 64, 329.63, 0], ['F4', 65, 349.23, 0], ['Fs1/Gb4', 66, 369.99, 0], ['G4', 67, 392.0, 0], ['Gs1/Ab4', 68, 415.3, 0], ['A4', 69, 440.0, 0], ['As1/Bb4', 70, 466.16, 0], ['B4', 71, 493.88, 0], ['C5', 72, 523.25, 0], ['Cs1/Db5', 73, 554.37, 0], ['D5', 74, 587.33, 0], ['Ds1/Eb5', 75, 622.25, 0], ['E5', 76, 659.26, 0], ['F5', 77, 698.46, 0], ['Fs1/Gb5', 78, 739.99, 0], ['G5', 79, 783.99, 0], ['Gs1/Ab5', 80, 830.61, 0], ['A5', 81, 880.0, 0], ['As1/Bb5', 82, 932.33, 0], ['B5', 83, 987.77, 0], ['C6', 84, 1046.5, 0], ['Cs1/Db6', 85, 1108.73, 0], ['D6', 86, 1174.66, 0], ['Ds1/Eb6', 87, 1244.51, 0], ['E6', 88, 1318.51, 0], ['F6', 89, 1396.91, 0], ['Fs1/Gb6', 90, 1479.98, 0], ['G6', 91, 1567.98, 0], ['Gs1/Ab6', 92, 1661.22, 0], ['A6', 93, 1760.0, 0], ['As1/Bb6', 94, 1864.66, 0], ['B6', 95, 1975.53, 0], ['C7', 96, 2093.0, 0], ['Cs1/Db7', 97, 2217.46, 0], ['D7', 98, 2349.32, 0], ['Ds1/Eb7', 99, 2489.02, 0], ['E7', 100, 2637.02, 0], ['F7', 101, 2793.83, 0], ['Fs1/Gb7', 102, 2959.96, 0], ['G7', 103, 3135.96, 0], ['Gs1/Ab7', 104, 3322.44, 0], ['A7', 105, 3520.0, 0], ['As1/Bb7', 106, 3729.31, 0], ['B7', 107, 3951.07, 0], ['C8', 108, 4186.01, 0]]
    # This is a VERY long list of all notes on Piano. While yes, I could have kept this in a .csv file or even put this into a dictionary,
    # I wanted to have it clearly written in front of me.
    # This also makes it easier to edit and/or change the list as necessary.
    # FORMAT OF LIST: [note, MIDI, frequency, amplitude]
    # Amplitude will change depending on what will be read from the .WAV file.


    for i in range(0,len(possible_tones)):
        f = possible_tones[i][2]
        # This is the frequency of the tone in the table.
        # Even though the sound from the file may not reach this perfect frequency, each MIDI's frequency is perfect, so
        # analysis of the sound needs to be done to find the most prominent perfect frequency.
        for j in range(to_cut_off-1):
            f1 = xf[j] #point on graph x-axis to analyse
            f2 = xf[j+1] #one ahead of point f1
            if  (f1 <= f < f2):
                A = abs(abs(yf[j+1]) - abs(yf[j]))/(f2-f1)+abs(yf[j])
                if A > Amax:
                    Amax = A
                    jmax = i
                possible_tones[i][3] = A
                break    
    # Method behind this:
    # 
    # See OneNote for more in-depth description
#------------------------------------------------------------------------------------
    
    p = len(possible_tones)
    for pp in range(len(possible_tones)):
        if Amax != 0:
            possible_tones[pp][3] = possible_tones[pp][3]/Amax*100
            if possible_tones[pp][3] <=10:
                # Tones less than 10 power/ amplitude are filtered out -- tone set to 1 as negligible sound.
                possible_tones[pp][3] = 1
        else:
            possible_tones[pp][3] = 1



##INSTRUMENT ROUTINE
    for tn in possible_tones:
        if tn[3] > 1: #finds the first tone with volume above 10
            maxko = tn[0]
            m = tn[1]
            maxvol = tn[3]
            break
    
##    print("m strategy first above 10 tone: ",m,maxvol)
##    input()
###HUMAN VOICE  ROUTINE
    # if tonestype == "human":
    # m = possible_tones[jmax][1] #this is max volume tone in the sample - good for HUMAN voice identification - not for instrument
    # maxvol = possible_tones[jmax][3]

    for elem in possible_tones:
        print(elem)
    return m
#####FUNCTION WHICH RETURNS SINGLE MIDI FOR SINGLE TONE FILE - END

single_pitch_to_MIDI(normalised_tone,rate)

# ####FUNCTION FOR RETURNING WRITTEN TONE FROM MIDI VALUE
# def findthetonetomidi(midivalue):
#     tone=[['A0', 'A', 0, 27.5, 21, 0], ['Bb0', 'Bb', 0, 29.135, 22, 0], ['B0', 'B', 0, 30.868, 23, 0], ['C1', 'C', 1, 32.703, 24, 0], ['Cs1', 'Cs', 1, 34.648, 25, 0], ['D1', 'D', 1, 36.708, 26, 0], ['Eb1', 'Eb', 1, 38.891, 27, 0], ['E1', 'E', 1, 41.203, 28, 0], ['F1', 'F', 1, 43.654, 29, 0], ['Fs1', 'Fs', 1, 46.249, 30, 0], ['G1', 'G', 1, 48.999, 31, 0], ['Gs1', 'Gs', 1, 51.913, 32, 0], ['A1', 'A', 1, 55, 33, 0], ['Bb1', 'Bb', 1, 58.27, 34, 0], ['B1', 'B', 1, 61.735, 35, 0], ['C2', 'C', 2, 65.406, 36, 0], ['Cs2', 'Cs', 2, 69.296, 37, 0], ['D2', 'D', 2, 73.416, 38, 0], ['Eb2', 'Eb', 2, 77.782, 39, 0], ['E2', 'E', 2, 82.407, 40, 0], ['F2', 'F', 2, 87.307, 41, 0], ['Fs2', 'Fs', 2, 92.499, 42, 0], ['G2', 'G', 2, 97.999, 43, 0], ['Gs2', 'Gs', 2, 103.83, 44, 0], ['A2', 'A', 2, 110, 45, 0], ['Bb2', 'Bb', 2, 116.54, 46, 0], ['B2', 'B', 2, 123.47, 47, 0], ['C3', 'C', 3, 130.81, 48, 0], ['Cs3', 'Cs', 3, 138.59, 49, 0], ['D3', 'D', 3, 146.83, 50, 0], ['Eb3', 'Eb', 3, 155.56, 51, 0], ['E3', 'E', 3, 164.81, 52, 0], ['F3', 'F', 3, 174.61, 53, 0], ['Fs3', 'Fs', 3, 185, 54, 0], ['G3', 'G', 3, 196, 55, 0], ['Gs3', 'Gs', 3, 207.65, 56, 0], ['A3', 'A', 3, 220, 57, 0], ['Bb3', 'Bb', 3, 233.08, 58, 0], ['B3', 'B', 3, 246.94, 59, 0], ['C4', 'C', 4, 261.63, 60, 0], ['Cs4', 'Cs', 4, 277.18, 61, 0], ['D4', 'D', 4, 293.66, 62, 0], ['Eb4', 'Eb', 4, 311.13, 63, 0], ['E4', 'E', 4, 329.63, 64, 0], ['F4', 'F', 4, 349.23, 65, 0], ['Fs4', 'Fs', 4, 369.99, 66, 0], ['G4', 'G', 4, 392, 67, 0], ['Gs4', 'Gs', 4, 415.3, 68, 0], ['A4', 'A', 4, 440, 69, 0], ['Bb4', 'Bb', 4, 466.16, 70, 0], ['B4', 'B', 4, 493.88, 71, 0], ['C5', 'C', 5, 523.25, 72, 0], ['Cs5', 'Cs', 5, 554.37, 73, 0], ['D5', 'D', 5, 587.33, 74, 0], ['Eb5', 'Eb', 5, 622.25, 75, 0], ['E5', 'E', 5, 659.26, 76, 0], ['F5', 'F', 5, 698.46, 77, 0], ['Fs5', 'Fs', 5, 739.99, 78, 0], ['G5', 'G', 5, 783.99, 79, 0], ['Gs5', 'Gs', 5, 830.61, 80, 0], ['A5', 'A', 5, 880, 81, 0], ['Bb5', 'Bb', 5, 932.33, 82, 0], ['B5', 'B', 5, 987.77, 83, 0], ['C6', 'C', 6, 1046.5, 84, 0], ['Cs6', 'Cs', 6, 1108.73, 85, 0], ['D6', 'D', 6, 1174.66, 86, 0], ['Eb6', 'Eb', 6, 1244.51, 87, 0], ['E6', 'E', 6, 1318.51, 88, 0], ['F6', 'F', 6, 1396.91, 89, 0], ['Fs6', 'Fs', 6, 1479.98, 90, 0], ['G6', 'G', 6, 1567.98, 91, 0], ['Gs6', 'Gs', 6, 1661.22, 92, 0], ['A6', 'A', 6, 1760, 93, 0], ['Bb6', 'Bb', 6, 1864.66, 94, 0], ['B6', 'B', 6, 1975.53, 95, 0], ['C7', 'C', 7, 2093, 96, 0], ['Cs7', 'Cs', 7, 2217.46, 97, 0], ['D7', 'D', 7, 2349.32, 98, 0], ['Eb7', 'Eb', 7, 2489.02, 99, 0], ['E7', 'E', 7, 2637.02, 100, 0], ['F7', 'F', 7, 2793.83, 101, 0], ['Fs7', 'Fs', 7, 2959.96, 102, 0], ['G7', 'G', 7, 3135.96, 103, 0], ['Gs7', 'Gs', 7, 3322.44, 104, 0], ['A7', 'A', 7, 3520, 105, 0], ['Bb7', 'Bb', 7, 3729.31, 106, 0], ['B7', 'B', 7, 3951.07, 107, 0], ['C8', 'C', 8, 4186.01, 108, 0]]
#     miditone=0
#     for t in range(0,len(tone)):
#             if tone[t][4] == midivalue:
#                 miditone = tone[t][0]
#                 break
#     return(miditone)
# ####FUNCTION FOR RETURNING TONE FROM MIDI VALUE

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

READING TONE FROM ONE FILE -- HUMAN VOICE

In [45]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import numpy as np

from scipy.fft import rfft, rfftfreq

in_wav = "C:\$$$AUDIO FILES\VOICE_C3.wav"

### STEP ONE: GET RID OF ONE CHANNEL IF ORIGINAL RECORDING HAS TWO SO EASIER TO DEAL WITH + NORMALISE FILE 
rate, data = wavfile.read(in_wav)
if data.ndim == 2: #data.ndim returns whether 1D or 2D list, i.e. 1 channel or 2 channels
    normalised_tone = np.int16((data[:,0]/data[:,0].max()) * 32767)
else:
    normalised_tone = np.int16((data[:]/data[:].max()) * 32767)

### STEP 2: HOW TO GET TONE OF WHOLE FILE -- ONLY FOR ONE PITCH FILE

def single_pitch_to_MIDI(normalised_tone,rate):
    yf = rfft(normalised_tone) 
    xf = rfftfreq(len(normalised_tone),1/rate) 

    points_per_freq = len(xf) / (rate/2)
    # This is the number of points plotted per 1Hz of frequency.
    # The maximum frequency possibly reached is half the sample rate
    to_cut_off = int(points_per_freq * 4200) # Sets up point to cut sound off at 4200Hz because piano's highest key is 4186Hz

    yf = yf[:to_cut_off+1] # Cut off all points above 4200 Hz
    xf = xf[:to_cut_off+1]



    A = 0
    Amax = 0
    possible_tones = [['A0', 21, 27.5, 0], ['As0/Bb0', 22, 29.14, 0], ['B0', 23, 30.87, 0], ['C1', 24, 32.7, 0], ['Cs1/Db1', 25, 34.65, 0], ['D1', 26, 36.71, 0], ['Ds1/Eb1', 27, 38.89, 0], ['E1', 28, 41.2, 0], ['F1', 29, 43.65, 0], ['Fs1/Gb1', 30, 46.25, 0], ['G1', 31, 49.0, 0], ['Gs1/Ab1', 32, 51.91, 0], ['A1', 33, 55.0, 0], ['As1/Bb1', 34, 58.27, 0], ['B1', 35, 61.74, 0], ['C2', 36, 65.41, 0], ['Cs1/Db2', 37, 69.3, 0], ['D2', 38, 73.42, 0], ['Ds1/Eb2', 39, 77.78, 0], ['E2', 40, 82.41, 0], ['F2', 41, 87.31, 0], ['Fs1/Gb2', 42, 92.5, 0], ['G2', 43, 98.0, 0], ['Gs1/Ab2', 44, 103.83, 0], ['A2', 45, 110.0, 0], ['As1/Bb2', 46, 116.54, 0], ['B2', 47, 123.47, 0], ['C3', 48, 130.81, 0], ['Cs1/Db3', 49, 138.59, 0], ['D3', 50, 146.83, 0], ['Ds1/Eb3', 51, 155.56, 0], ['E3', 52, 164.81, 0], ['F3', 53, 174.61, 0], ['Fs1/Gb3', 54, 185.0, 0], ['G3', 55, 196.0, 0], ['Gs1/Ab3', 56, 207.65, 0], ['A3', 57, 220.0, 0], ['As1/Bb3', 58, 233.08, 0], ['B3', 59, 246.94, 0], ['C4', 60, 261.63, 0], ['Cs1/Db4', 61, 277.18, 0], ['D4', 62, 293.66, 0], ['Ds1/Eb4', 63, 311.13, 0], ['E4', 64, 329.63, 0], ['F4', 65, 349.23, 0], ['Fs1/Gb4', 66, 369.99, 0], ['G4', 67, 392.0, 0], ['Gs1/Ab4', 68, 415.3, 0], ['A4', 69, 440.0, 0], ['As1/Bb4', 70, 466.16, 0], ['B4', 71, 493.88, 0], ['C5', 72, 523.25, 0], ['Cs1/Db5', 73, 554.37, 0], ['D5', 74, 587.33, 0], ['Ds1/Eb5', 75, 622.25, 0], ['E5', 76, 659.26, 0], ['F5', 77, 698.46, 0], ['Fs1/Gb5', 78, 739.99, 0], ['G5', 79, 783.99, 0], ['Gs1/Ab5', 80, 830.61, 0], ['A5', 81, 880.0, 0], ['As1/Bb5', 82, 932.33, 0], ['B5', 83, 987.77, 0], ['C6', 84, 1046.5, 0], ['Cs1/Db6', 85, 1108.73, 0], ['D6', 86, 1174.66, 0], ['Ds1/Eb6', 87, 1244.51, 0], ['E6', 88, 1318.51, 0], ['F6', 89, 1396.91, 0], ['Fs1/Gb6', 90, 1479.98, 0], ['G6', 91, 1567.98, 0], ['Gs1/Ab6', 92, 1661.22, 0], ['A6', 93, 1760.0, 0], ['As1/Bb6', 94, 1864.66, 0], ['B6', 95, 1975.53, 0], ['C7', 96, 2093.0, 0], ['Cs1/Db7', 97, 2217.46, 0], ['D7', 98, 2349.32, 0], ['Ds1/Eb7', 99, 2489.02, 0], ['E7', 100, 2637.02, 0], ['F7', 101, 2793.83, 0], ['Fs1/Gb7', 102, 2959.96, 0], ['G7', 103, 3135.96, 0], ['Gs1/Ab7', 104, 3322.44, 0], ['A7', 105, 3520.0, 0], ['As1/Bb7', 106, 3729.31, 0], ['B7', 107, 3951.07, 0], ['C8', 108, 4186.01, 0]]
    # This is a VERY long list of all notes on Piano. While yes, I could have kept this in a .csv file or even put this into a dictionary,
    # I wanted to have it clearly written in front of me.
    # This also makes it easier to edit and/or change the list as necessary.
    # FORMAT OF LIST: [note, MIDI, frequency, amplitude]
    # Amplitude will change depending on what will be read from the .WAV file.


    for i in range(0,len(possible_tones)):
        f = possible_tones[i][2]
        # This is the frequency of the tone in the table.
        # Even though the sound from the file may not reach this perfect frequency, each MIDI's frequency is perfect, so
        # analysis of the sound needs to be done to find the most prominent perfect frequency.
        for j in range(to_cut_off-1):
            f1 = xf[j] #point on graph x-axis to analyse
            f2 = xf[j+1] #one ahead of point f1
            if  (f1 <= f < f2):
                A = abs(abs(yf[j+1]) - abs(yf[j]))/(f2-f1)+abs(yf[j])
                if A > Amax:
                    Amax = A
                    jmax = i
                possible_tones[i][3] = A
                break    
    # Method behind this:
    # 
    # See OneNote for more in-depth description
#------------------------------------------------------------------------------------
    
    p = len(possible_tones)
    for pp in range(len(possible_tones)):
        if Amax != 0:
            possible_tones[pp][3] = possible_tones[pp][3]/Amax*100
            if possible_tones[pp][3] <=10:
                # Tones less than 10 power/ amplitude are filtered out -- tone set to 1 as negligible sound.
                possible_tones[pp][3] = 1
        else:
            possible_tones[pp][3] = 1



##INSTRUMENT ROUTINE
    for tn in possible_tones:
        if tn[3] > 1: #finds the first tone with volume above 10
            maxko = tn[0]
            m = tn[1]
            maxvol = tn[3]
            break
    
##    print("m strategy first above 10 tone: ",m,maxvol)
##    input()
###HUMAN VOICE  ROUTINE
    # if tonestype == "human":
    # m = possible_tones[jmax][1] #this is max volume tone in the sample - good for HUMAN voice identification - not for instrument
    # maxvol = possible_tones[jmax][3]

    for elem in possible_tones:
        print(elem)
    return m
#####FUNCTION WHICH RETURNS SINGLE MIDI FOR SINGLE TONE FILE - END

single_pitch_to_MIDI(normalised_tone,rate)

# ####FUNCTION FOR RETURNING WRITTEN TONE FROM MIDI VALUE
# def findthetonetomidi(midivalue):
#     tone=[['A0', 'A', 0, 27.5, 21, 0], ['Bb0', 'Bb', 0, 29.135, 22, 0], ['B0', 'B', 0, 30.868, 23, 0], ['C1', 'C', 1, 32.703, 24, 0], ['Cs1', 'Cs', 1, 34.648, 25, 0], ['D1', 'D', 1, 36.708, 26, 0], ['Eb1', 'Eb', 1, 38.891, 27, 0], ['E1', 'E', 1, 41.203, 28, 0], ['F1', 'F', 1, 43.654, 29, 0], ['Fs1', 'Fs', 1, 46.249, 30, 0], ['G1', 'G', 1, 48.999, 31, 0], ['Gs1', 'Gs', 1, 51.913, 32, 0], ['A1', 'A', 1, 55, 33, 0], ['Bb1', 'Bb', 1, 58.27, 34, 0], ['B1', 'B', 1, 61.735, 35, 0], ['C2', 'C', 2, 65.406, 36, 0], ['Cs2', 'Cs', 2, 69.296, 37, 0], ['D2', 'D', 2, 73.416, 38, 0], ['Eb2', 'Eb', 2, 77.782, 39, 0], ['E2', 'E', 2, 82.407, 40, 0], ['F2', 'F', 2, 87.307, 41, 0], ['Fs2', 'Fs', 2, 92.499, 42, 0], ['G2', 'G', 2, 97.999, 43, 0], ['Gs2', 'Gs', 2, 103.83, 44, 0], ['A2', 'A', 2, 110, 45, 0], ['Bb2', 'Bb', 2, 116.54, 46, 0], ['B2', 'B', 2, 123.47, 47, 0], ['C3', 'C', 3, 130.81, 48, 0], ['Cs3', 'Cs', 3, 138.59, 49, 0], ['D3', 'D', 3, 146.83, 50, 0], ['Eb3', 'Eb', 3, 155.56, 51, 0], ['E3', 'E', 3, 164.81, 52, 0], ['F3', 'F', 3, 174.61, 53, 0], ['Fs3', 'Fs', 3, 185, 54, 0], ['G3', 'G', 3, 196, 55, 0], ['Gs3', 'Gs', 3, 207.65, 56, 0], ['A3', 'A', 3, 220, 57, 0], ['Bb3', 'Bb', 3, 233.08, 58, 0], ['B3', 'B', 3, 246.94, 59, 0], ['C4', 'C', 4, 261.63, 60, 0], ['Cs4', 'Cs', 4, 277.18, 61, 0], ['D4', 'D', 4, 293.66, 62, 0], ['Eb4', 'Eb', 4, 311.13, 63, 0], ['E4', 'E', 4, 329.63, 64, 0], ['F4', 'F', 4, 349.23, 65, 0], ['Fs4', 'Fs', 4, 369.99, 66, 0], ['G4', 'G', 4, 392, 67, 0], ['Gs4', 'Gs', 4, 415.3, 68, 0], ['A4', 'A', 4, 440, 69, 0], ['Bb4', 'Bb', 4, 466.16, 70, 0], ['B4', 'B', 4, 493.88, 71, 0], ['C5', 'C', 5, 523.25, 72, 0], ['Cs5', 'Cs', 5, 554.37, 73, 0], ['D5', 'D', 5, 587.33, 74, 0], ['Eb5', 'Eb', 5, 622.25, 75, 0], ['E5', 'E', 5, 659.26, 76, 0], ['F5', 'F', 5, 698.46, 77, 0], ['Fs5', 'Fs', 5, 739.99, 78, 0], ['G5', 'G', 5, 783.99, 79, 0], ['Gs5', 'Gs', 5, 830.61, 80, 0], ['A5', 'A', 5, 880, 81, 0], ['Bb5', 'Bb', 5, 932.33, 82, 0], ['B5', 'B', 5, 987.77, 83, 0], ['C6', 'C', 6, 1046.5, 84, 0], ['Cs6', 'Cs', 6, 1108.73, 85, 0], ['D6', 'D', 6, 1174.66, 86, 0], ['Eb6', 'Eb', 6, 1244.51, 87, 0], ['E6', 'E', 6, 1318.51, 88, 0], ['F6', 'F', 6, 1396.91, 89, 0], ['Fs6', 'Fs', 6, 1479.98, 90, 0], ['G6', 'G', 6, 1567.98, 91, 0], ['Gs6', 'Gs', 6, 1661.22, 92, 0], ['A6', 'A', 6, 1760, 93, 0], ['Bb6', 'Bb', 6, 1864.66, 94, 0], ['B6', 'B', 6, 1975.53, 95, 0], ['C7', 'C', 7, 2093, 96, 0], ['Cs7', 'Cs', 7, 2217.46, 97, 0], ['D7', 'D', 7, 2349.32, 98, 0], ['Eb7', 'Eb', 7, 2489.02, 99, 0], ['E7', 'E', 7, 2637.02, 100, 0], ['F7', 'F', 7, 2793.83, 101, 0], ['Fs7', 'Fs', 7, 2959.96, 102, 0], ['G7', 'G', 7, 3135.96, 103, 0], ['Gs7', 'Gs', 7, 3322.44, 104, 0], ['A7', 'A', 7, 3520, 105, 0], ['Bb7', 'Bb', 7, 3729.31, 106, 0], ['B7', 'B', 7, 3951.07, 107, 0], ['C8', 'C', 8, 4186.01, 108, 0]]
#     miditone=0
#     for t in range(0,len(tone)):
#             if tone[t][4] == midivalue:
#                 miditone = tone[t][0]
#                 break
#     return(miditone)
# ####FUNCTION FOR RETURNING TONE FROM MIDI VALUE

['A0', 21, 27.5, 1]
['As0/Bb0', 22, 29.14, 1]
['B0', 23, 30.87, 1]
['C1', 24, 32.7, 1]
['Cs1/Db1', 25, 34.65, 1]
['D1', 26, 36.71, 1]
['Ds1/Eb1', 27, 38.89, 1]
['E1', 28, 41.2, 1]
['F1', 29, 43.65, 1]
['Fs1/Gb1', 30, 46.25, 1]
['G1', 31, 49.0, 1]
['Gs1/Ab1', 32, 51.91, 1]
['A1', 33, 55.0, 1]
['As1/Bb1', 34, 58.27, 1]
['B1', 35, 61.74, 1]
['C2', 36, 65.41, 1]
['Cs1/Db2', 37, 69.3, 1]
['D2', 38, 73.42, 1]
['Ds1/Eb2', 39, 77.78, 1]
['E2', 40, 82.41, 1]
['F2', 41, 87.31, 1]
['Fs1/Gb2', 42, 92.5, 1]
['G2', 43, 98.0, 1]
['Gs1/Ab2', 44, 103.83, 1]
['A2', 45, 110.0, 1]
['As1/Bb2', 46, 116.54, 1]
['B2', 47, 123.47, 1]
['C3', 48, 130.81, 1]
['Cs1/Db3', 49, 138.59, 1]
['D3', 50, 146.83, 1]
['Ds1/Eb3', 51, 155.56, 1]
['E3', 52, 164.81, 1]
['F3', 53, 174.61, 1]
['Fs1/Gb3', 54, 185.0, 1]
['G3', 55, 196.0, 1]
['Gs1/Ab3', 56, 207.65, 1]
['A3', 57, 220.0, 1]
['As1/Bb3', 58, 233.08, 1]
['B3', 59, 246.94, 1]
['C4', 60, 261.63, 100.0]
['Cs1/Db4', 61, 277.18, 1]
['D4', 62, 293.66, 1]
['Ds1/Eb4', 63, 311.13

60

Upon further inspection -- when I used human voice, the most powerful/ highest amplitude frequency was the frequency that corresponded to the right note, while with the piano I had to force the note down to the first note that had a high amplitude.

Due to different sound envelopes, the piano has different and more powerful overtones/ harmonics compared to the human voice.

Thus, now I have two methods -- one for piano, and one for human voice.

In [ ]:
print("Please wait till the next message. Libraries are loading now")

import scipy, matplotlib
#https://realpython.com/python-scipy-fft/ the gude was from here
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import wavfile
import scipy.io
from scipy.io.wavfile import write
from midiutil import MIDIFile
import math
from scipy.fft import irfft
from pygame import mixer
from scipy.fft import rfft, rfftfreq #import Frourier Transformation library
from basic_pitch.inference import predict
from basic_pitch.inference import predict_and_save
from basic_pitch import ICASSP_2022_MODEL_PATH
import os
import pathlib
import tempfile
import unittest
import librosa
import pretty_midi
from basic_pitch import ICASSP_2022_MODEL_PATH, inference
from basic_pitch.constants import ANNOTATIONS_N_SEMITONES

#https://github.com/craffel/pretty-midi/blob/main/Tutorial.ipynb
# For Python2.6 compatibility
#from __future__ import print_function
import pretty_midi
import numpy as np
# For plotting
import mir_eval.display
import librosa.display
import matplotlib.pyplot as plt
#%matplotlib inline
# For putting audio in the notebook
import IPython.display

import sounddevice as sd
from scipy.io.wavfile import write

import pyaudio
import wave
import time
#https://www.programiz.com/python-programming/time#:~:text=In%20Python%2C%20the%20time(),00%20at%20UTC%20is%20epoch.&text=In%20the%20above%20example%2C%20we,and%20then%20printed%20the%20result.

print("Libraries have been loaded")


##########################################record voice by laptop microphone
chunk = 512#1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 5
outwav = "C:\F\AIVO Katie\$$Wav to Midi\OUTPUT.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print("Recording for:  XXX seconds after you input number of seconds and press enter key. Default is: ",seconds,"seconds")
seconds = int(input("Input number of seconds: "))
print("Recording for: ",seconds)

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for X seconds
s0 = time.time()
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)
    print(i,time.time()-s0)


# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')
# Save the recorded data as a WAV file
wf = wave.open(outwav, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()
print("Finished Saving WAV FLLE to: ", filename)
input()
##########################################record voice by laptop microphone


##https://github.com/spotify/basic-pitch
#https://basicpitch.spotify.com/ = this is WRB SITE DROP IN
##predict_and_save(
##    <input-audio-path-list>,
##    <output-directory>,
##    <save-midi>,
##    <sonify-midi>,
##    <save-model-outputs>,
##    <save-note-events>,
##)
##predict()
##
##Import basic-pitch into your own Python code and run the predict functions directly, providing an <input-audio-path> and returning the model's prediction results:
##
##from basic_pitch.inference import predict
##from basic_pitch import ICASSP_2022_MODEL_PATH
##
##model_output, midi_data, note_activations = predict(<input-audio-path>)
##<minimum-frequency> & <maximum-frequency> (floats) set the maximum and minimum allowed note frequency, in Hz, returned by the model. Pitch events with frequencies outside of this range will be excluded from the prediction results.
##model_output is the raw model inference output
##midi_data is the transcribed MIDI data derived from the model_output
##note_events is a list of note events derived from the model_output





outmidishortanalysis = inwav[:len(inwav)-4]+"ShortAnalysis.mid"
outmidibeatanalysis = inwav[:len(inwav)-4]+"BeatAnalysis.mid"
outmidilonganalysis = inwav[:len(inwav)-4]+"LongAnalysis.mid"
outmidilonganalysisoriginal = inwav[:len(inwav)-4]+"LongAnalysisOriginal.mid"
outmidibasicpitch = inwav[:len(inwav)-4]+"Basicpitch.mid"
outmidibasicpitchoriginal = inwav[:len(inwav)-4]+"Basicpitchoriginal.mid" #as per Spotify script

mixer.init()

###THESE ARE MOST IMPORTNAT SWITCHES
inwav = outwav # recorgin of voice
BPM = 60 #BPM is very important and it is major input to the process. BPM = 60 works quite well with tone recognition;  BPM = 120 is used for most classical music
tonestype = "human"
#tonestype = "instrument"
###THESE ARE MOST IMPORTNAT SWITCHES


#####MIXER AND PLAYER FUNCTION DEFINED - BEGINING
def playmusicfile(inwav):
        #Load audio file
    mixer.music.load(inwav)
    print("music started playing for file called: ", inwav)
    #Set preferred volume
    mixer.music.set_volume(0.5)
    #Play the music
    mixer.music.play(-1) #-1 means infitinte loop
    #Infinite loop
    while True:
        print("We are playing file: ",inwav)
        print("Press 'p' to pause the music")
        print("Press 'r' to resume the music")
        print("Press 'e' to exit the program")
        #take user input
        userInput = input(" ")
        if userInput == 'p':
            # Pause the music
            mixer.music.pause()
            print("music is paused....")
        elif userInput == 'r':
            #Resume the music
            mixer.music.unpause()
            print("music is resumed....")
        elif userInput == 'e':
            # Stop the music playback
            mixer.music.stop()
            print("music is stopped....")
            break
#####MIXER AND PLAYER FUNCTION DEFINED - END


        

#####FUNCTION WHICH RETURNS SINGLE MIDI FOR SINGLE TONE VAW FILE - BEGINING
def toneandmidiforsinglewavfile(normalized_tone_left_samples,samplerate,tonestype):
    N=len(normalized_tone_left_samples)
    yf = rfft(normalized_tone_left_samples)
    xf = rfftfreq(N, 1 / samplerate)
    k = min(len(xf),len(yf))
    xf = xf[:k]
    yf=yf[:k]
    points_per_freq = len(xf) / (samplerate / 2) #this is number of x plotted points for 1 Herz of frequency
    # Our target frequency is 4000 Hz for showing on the graph; Piano highest key is: 4186 Hz
    k = int(points_per_freq * 4200) #maximum 4200 Hz
    xf = xf[:k] #all frequencies above 4200 Hz is cut off
    yf=yf[:k]
    k0 = 0
    A = 0
    Amax = 0
    tone=[['A0', 'A', 0, 27.5, 21, 0], ['Bb0', 'Bb', 0, 29.135, 22, 0], ['B0', 'B', 0, 30.868, 23, 0], ['C1', 'C', 1, 32.703, 24, 0], ['Cs1', 'Cs', 1, 34.648, 25, 0], ['D1', 'D', 1, 36.708, 26, 0], ['Eb1', 'Eb', 1, 38.891, 27, 0], ['E1', 'E', 1, 41.203, 28, 0], ['F1', 'F', 1, 43.654, 29, 0], ['Fs1', 'Fs', 1, 46.249, 30, 0], ['G1', 'G', 1, 48.999, 31, 0], ['Gs1', 'Gs', 1, 51.913, 32, 0], ['A1', 'A', 1, 55, 33, 0], ['Bb1', 'Bb', 1, 58.27, 34, 0], ['B1', 'B', 1, 61.735, 35, 0], ['C2', 'C', 2, 65.406, 36, 0], ['Cs2', 'Cs', 2, 69.296, 37, 0], ['D2', 'D', 2, 73.416, 38, 0], ['Eb2', 'Eb', 2, 77.782, 39, 0], ['E2', 'E', 2, 82.407, 40, 0], ['F2', 'F', 2, 87.307, 41, 0], ['Fs2', 'Fs', 2, 92.499, 42, 0], ['G2', 'G', 2, 97.999, 43, 0], ['Gs2', 'Gs', 2, 103.83, 44, 0], ['A2', 'A', 2, 110, 45, 0], ['Bb2', 'Bb', 2, 116.54, 46, 0], ['B2', 'B', 2, 123.47, 47, 0], ['C3', 'C', 3, 130.81, 48, 0], ['Cs3', 'Cs', 3, 138.59, 49, 0], ['D3', 'D', 3, 146.83, 50, 0], ['Eb3', 'Eb', 3, 155.56, 51, 0], ['E3', 'E', 3, 164.81, 52, 0], ['F3', 'F', 3, 174.61, 53, 0], ['Fs3', 'Fs', 3, 185, 54, 0], ['G3', 'G', 3, 196, 55, 0], ['Gs3', 'Gs', 3, 207.65, 56, 0], ['A3', 'A', 3, 220, 57, 0], ['Bb3', 'Bb', 3, 233.08, 58, 0], ['B3', 'B', 3, 246.94, 59, 0], ['C4', 'C', 4, 261.63, 60, 0], ['Cs4', 'Cs', 4, 277.18, 61, 0], ['D4', 'D', 4, 293.66, 62, 0], ['Eb4', 'Eb', 4, 311.13, 63, 0], ['E4', 'E', 4, 329.63, 64, 0], ['F4', 'F', 4, 349.23, 65, 0], ['Fs4', 'Fs', 4, 369.99, 66, 0], ['G4', 'G', 4, 392, 67, 0], ['Gs4', 'Gs', 4, 415.3, 68, 0], ['A4', 'A', 4, 440, 69, 0], ['Bb4', 'Bb', 4, 466.16, 70, 0], ['B4', 'B', 4, 493.88, 71, 0], ['C5', 'C', 5, 523.25, 72, 0], ['Cs5', 'Cs', 5, 554.37, 73, 0], ['D5', 'D', 5, 587.33, 74, 0], ['Eb5', 'Eb', 5, 622.25, 75, 0], ['E5', 'E', 5, 659.26, 76, 0], ['F5', 'F', 5, 698.46, 77, 0], ['Fs5', 'Fs', 5, 739.99, 78, 0], ['G5', 'G', 5, 783.99, 79, 0], ['Gs5', 'Gs', 5, 830.61, 80, 0], ['A5', 'A', 5, 880, 81, 0], ['Bb5', 'Bb', 5, 932.33, 82, 0], ['B5', 'B', 5, 987.77, 83, 0], ['C6', 'C', 6, 1046.5, 84, 0], ['Cs6', 'Cs', 6, 1108.73, 85, 0], ['D6', 'D', 6, 1174.66, 86, 0], ['Eb6', 'Eb', 6, 1244.51, 87, 0], ['E6', 'E', 6, 1318.51, 88, 0], ['F6', 'F', 6, 1396.91, 89, 0], ['Fs6', 'Fs', 6, 1479.98, 90, 0], ['G6', 'G', 6, 1567.98, 91, 0], ['Gs6', 'Gs', 6, 1661.22, 92, 0], ['A6', 'A', 6, 1760, 93, 0], ['Bb6', 'Bb', 6, 1864.66, 94, 0], ['B6', 'B', 6, 1975.53, 95, 0], ['C7', 'C', 7, 2093, 96, 0], ['Cs7', 'Cs', 7, 2217.46, 97, 0], ['D7', 'D', 7, 2349.32, 98, 0], ['Eb7', 'Eb', 7, 2489.02, 99, 0], ['E7', 'E', 7, 2637.02, 100, 0], ['F7', 'F', 7, 2793.83, 101, 0], ['Fs7', 'Fs', 7, 2959.96, 102, 0], ['G7', 'G', 7, 3135.96, 103, 0], ['Gs7', 'Gs', 7, 3322.44, 104, 0], ['A7', 'A', 7, 3520, 105, 0], ['Bb7', 'Bb', 7, 3729.31, 106, 0], ['B7', 'B', 7, 3951.07, 107, 0], ['C8', 'C', 8, 4186.01, 108, 0]]
    #['A0', 'A', 0, 27.5, 21, 0] = one record of keyboard; Item[5] is Amplitude of Fourier transformation
    kkey = [['A',1], ['Bb',1], ['B',1], ['C',1], ['Cs',1], ['D',1], ['Eb',1], ['E',1], ['F',1], ['Fs',1], ['G',1], ['Gs',1]]
    maxoctave = 0 
    t=len(tone)
    jmax = 0
    for j in range(0,t):
        f = tone[j][3] #frequency of the tone in the table - I am looking for values only for the Tones of keyboard (not for all frequencies)
        for i in range(k0,k-1):
            #i = i + 1
            f1 = xf[i]
            f2 = xf[i+1]
            if  ((f1 <= f) and (f < f2)):
                A = abs(abs(yf[i+1]) - abs(yf[i]))/(f2-f1)+abs(yf[i])
                if A > Amax:
                    Amax = A
                    jmax = j
                tone[j][5] = A
                k0 = k0+1
                break    
    p = len(tone)
    q = len(kkey)
    #r = len(octave)
    for pp in range(0,t):
        if Amax != 0:
            tone[pp][5] = tone[pp][5]/Amax*100
            if tone[pp][5] <=10: #to filter out smaller sounding tones - only 10 and more from 100 are allowed) - otherwise set to 1
                tone[pp][5] = 1
        else: tone[pp][5] = 1


##    for t in tone:
##        print(t)
##    input()
####
##INSTRUMENT ROUTINE
    for tn in tone:
        if tn[5] > 1: #finds the first tone with volume above 10 - which is probably low cut out
            maxko = tn[0]
            m = tn[4]
            maxvol = tn[5]
            break
    
##    print("m strategy first above 10 tone: ",m,maxvol)
##    input()
###HUMAN VOICE  ROUTINE
    if tonestype == "human":
        m = tone[jmax][4] #this is max volume tone in the sample - good for HUMAN voice identification - not for instrument
        maxvol = tone[jmax][5]
##    print("m strategy max volume tone: ",m,maxvol)
##    input()
##    
    
           
##    print("maxko: ",maxko) #tone for maximum
##    print(m) #midi for maximum
    return(m)
#####FUNCTION WHICH RETURNS SINGLE MIDI FOR SINGLE TONE FILE - END

####FUNCTION FOR RETURNING WRITTEN TONE FROM MIDI VALUE
def findthetonetomidi(midivalue):
    tone=[['A0', 'A', 0, 27.5, 21, 0], ['Bb0', 'Bb', 0, 29.135, 22, 0], ['B0', 'B', 0, 30.868, 23, 0], ['C1', 'C', 1, 32.703, 24, 0], ['Cs1', 'Cs', 1, 34.648, 25, 0], ['D1', 'D', 1, 36.708, 26, 0], ['Eb1', 'Eb', 1, 38.891, 27, 0], ['E1', 'E', 1, 41.203, 28, 0], ['F1', 'F', 1, 43.654, 29, 0], ['Fs1', 'Fs', 1, 46.249, 30, 0], ['G1', 'G', 1, 48.999, 31, 0], ['Gs1', 'Gs', 1, 51.913, 32, 0], ['A1', 'A', 1, 55, 33, 0], ['Bb1', 'Bb', 1, 58.27, 34, 0], ['B1', 'B', 1, 61.735, 35, 0], ['C2', 'C', 2, 65.406, 36, 0], ['Cs2', 'Cs', 2, 69.296, 37, 0], ['D2', 'D', 2, 73.416, 38, 0], ['Eb2', 'Eb', 2, 77.782, 39, 0], ['E2', 'E', 2, 82.407, 40, 0], ['F2', 'F', 2, 87.307, 41, 0], ['Fs2', 'Fs', 2, 92.499, 42, 0], ['G2', 'G', 2, 97.999, 43, 0], ['Gs2', 'Gs', 2, 103.83, 44, 0], ['A2', 'A', 2, 110, 45, 0], ['Bb2', 'Bb', 2, 116.54, 46, 0], ['B2', 'B', 2, 123.47, 47, 0], ['C3', 'C', 3, 130.81, 48, 0], ['Cs3', 'Cs', 3, 138.59, 49, 0], ['D3', 'D', 3, 146.83, 50, 0], ['Eb3', 'Eb', 3, 155.56, 51, 0], ['E3', 'E', 3, 164.81, 52, 0], ['F3', 'F', 3, 174.61, 53, 0], ['Fs3', 'Fs', 3, 185, 54, 0], ['G3', 'G', 3, 196, 55, 0], ['Gs3', 'Gs', 3, 207.65, 56, 0], ['A3', 'A', 3, 220, 57, 0], ['Bb3', 'Bb', 3, 233.08, 58, 0], ['B3', 'B', 3, 246.94, 59, 0], ['C4', 'C', 4, 261.63, 60, 0], ['Cs4', 'Cs', 4, 277.18, 61, 0], ['D4', 'D', 4, 293.66, 62, 0], ['Eb4', 'Eb', 4, 311.13, 63, 0], ['E4', 'E', 4, 329.63, 64, 0], ['F4', 'F', 4, 349.23, 65, 0], ['Fs4', 'Fs', 4, 369.99, 66, 0], ['G4', 'G', 4, 392, 67, 0], ['Gs4', 'Gs', 4, 415.3, 68, 0], ['A4', 'A', 4, 440, 69, 0], ['Bb4', 'Bb', 4, 466.16, 70, 0], ['B4', 'B', 4, 493.88, 71, 0], ['C5', 'C', 5, 523.25, 72, 0], ['Cs5', 'Cs', 5, 554.37, 73, 0], ['D5', 'D', 5, 587.33, 74, 0], ['Eb5', 'Eb', 5, 622.25, 75, 0], ['E5', 'E', 5, 659.26, 76, 0], ['F5', 'F', 5, 698.46, 77, 0], ['Fs5', 'Fs', 5, 739.99, 78, 0], ['G5', 'G', 5, 783.99, 79, 0], ['Gs5', 'Gs', 5, 830.61, 80, 0], ['A5', 'A', 5, 880, 81, 0], ['Bb5', 'Bb', 5, 932.33, 82, 0], ['B5', 'B', 5, 987.77, 83, 0], ['C6', 'C', 6, 1046.5, 84, 0], ['Cs6', 'Cs', 6, 1108.73, 85, 0], ['D6', 'D', 6, 1174.66, 86, 0], ['Eb6', 'Eb', 6, 1244.51, 87, 0], ['E6', 'E', 6, 1318.51, 88, 0], ['F6', 'F', 6, 1396.91, 89, 0], ['Fs6', 'Fs', 6, 1479.98, 90, 0], ['G6', 'G', 6, 1567.98, 91, 0], ['Gs6', 'Gs', 6, 1661.22, 92, 0], ['A6', 'A', 6, 1760, 93, 0], ['Bb6', 'Bb', 6, 1864.66, 94, 0], ['B6', 'B', 6, 1975.53, 95, 0], ['C7', 'C', 7, 2093, 96, 0], ['Cs7', 'Cs', 7, 2217.46, 97, 0], ['D7', 'D', 7, 2349.32, 98, 0], ['Eb7', 'Eb', 7, 2489.02, 99, 0], ['E7', 'E', 7, 2637.02, 100, 0], ['F7', 'F', 7, 2793.83, 101, 0], ['Fs7', 'Fs', 7, 2959.96, 102, 0], ['G7', 'G', 7, 3135.96, 103, 0], ['Gs7', 'Gs', 7, 3322.44, 104, 0], ['A7', 'A', 7, 3520, 105, 0], ['Bb7', 'Bb', 7, 3729.31, 106, 0], ['B7', 'B', 7, 3951.07, 107, 0], ['C8', 'C', 8, 4186.01, 108, 0]]
    miditone=0
    for t in range(0,len(tone)):
            if tone[t][4] == midivalue:
                miditone = tone[t][0]
                break
    return(miditone)
####FUNCTION FOR RETURNING TONE FROM MIDI VALUE

#Initial description of thw WAV file
samplerate, data = wavfile.read(inwav)
print("samplerate datapoints per second: ",samplerate)
lengthofwavinseconds = data.shape[0] / samplerate #data.shape[0]is lenght of the wav file in seconds;
lengthofwavindatapoints = data.shape[0]
numberofchanels = data.ndim
print("length of wav in data points: ",lengthofwavindatapoints)
print("length of wav in seconds: ",lengthofwavinseconds)
print("numberofchanels: ",numberofchanels)
normalized_tone = np.int16((data[:]/data[:].max()) * 32767) # this is the tone definition normalizes to 32k bits = standard height for the tone to be identified#
playmusicfile(inwav)
plt.plot(normalized_tone)
plt.xlabel("X axis in points")
plt.ylabel("Amplitude")
plt.ylim([-32767, 32767])
plt.show()
#Normalized tone will be used for further analysis


lenghofsampleinseconds = 60/BPM #int(data.shape[0]/numberofsamples) in Seconds; if sample has lenght of 0.25 seconds = 240BPM (60 x 4)
print("BPM: ",BPM)
lenghofsampleindatapoints = int(lenghofsampleinseconds*samplerate) #int(data.shape[0]/numberofsamples) in datapoints in waw
print("lenghofsampleinseconds: ",lenghofsampleinseconds)
print("lenghofsampleindatapoints: ",lenghofsampleindatapoints)
##BPM


####Make noise between two distinct tones down to zero
#Important parameter is at what percentage of average sound or at what sound value  the sound should be cut off
normalized_tone_left = [0]*lengthofwavindatapoints #the list of averages
if numberofchanels == 1:
    for i in range(0,lengthofwavindatapoints):
        normalized_tone_left[i] = (normalized_tone[i])
else:
    for i in range(0,lengthofwavindatapoints):
        normalized_tone_left[i] = (normalized_tone[i][0]) #[0] = only left channel
        
for i in range(0,lengthofwavindatapoints):
    if abs(normalized_tone_left[i]) <1000: #filtering on 1000 out of 32000 - maybe can filter more
        normalized_tone_left[i] = 0
plt.plot(normalized_tone_left)
plt.xlabel("X axis in points")
plt.ylabel("Amplitude")
plt.ylim([-32767, 32767])
plt.show()
print("len(normalized_tone_left): ",len(normalized_tone_left))


###########################################MAIN_FUNCTION_FOR_GIVEN_SAMPLE_INTERVAL_Whis_is_0.01secondwide##################################################
def returnmidivalueforsamplefromdatapoint(data,s0,lenghofsampleindatapoints):
    s1 = s0 + lenghofsampleindatapoints
    #data1s = data[s0:s1,0] #0 means left channel of wav
    data1s = data[s0:s1] #this is from already left channel of normalized wav
    normalized_tone = data1s
#    if data1s[:].max() > 0:
#        normalized_tone = np.int16((data1s[:] / data1s[:].max()) * 32767) # this is the tone definition normalizes to 32k bits = standard height for the tone to be identified
##    plt.plot(s0,s1,normalized_tone)
##    plt.xlabel("X axis in points")
##    plt.ylabel("Amplitude")
##    plt.ylim([-32767, 32767])
##    plt.show()
    N = s1 - s0 #N -s niumber of points on X Axis (frequencies); length is the length of the Waw file in seconds
    yf = rfft(normalized_tone) #Rapid Fourier transformation on Normalized Tone - returns Aplitude for each frequency
    xf = rfftfreq(N, 1 / samplerate)
    # Return the Discrete Fourier Transform sample frequencies (for usage with rfft, irfft).
    #The returned float array f contains the frequency bin centers in cycles per unit of the sample spacing (with zero at the start).
    #For instance, if the sample spacing is in seconds, then the frequency unit is cycles/second.
    #Sample spacing (inverse of the sampling rate). Defaults to 1.
    #print("Plot of FFT Frequency distribution - this meanst FFT worked")
    k = min(len(xf),len(yf))
##    print("len(xf),len(yf),k:",len(xf),len(yf),k)
##    input()
    xf = xf[:k]
    yf=yf[:k]
##    plt.plot(xf, np.abs(yf))
##    plt.xlabel("Frequencies in the Wave Sample in Hz")
##    plt.ylabel("FFT Amplitude of given Frequency")
##    plt.xlim([0,4000]) #plot frequency from 0 to 4000 Hz
##    plt.show()
    # The maximum detectable frequency is half the sample rate (44100 sample rate can determine frequency up to 22kHz (ie. maximum hearing range)

    fftpoint1 = 0
    fftyvalue1 =0
    fftpoint2 = 0
    fftyvalue2 =0
    fftpoint3 = 0
    fftyvalue3 =0
    fftpoint4 = 0
    fftyvalue4 =0
    fftpoint5 = 0
    fftyvalue5 =0
    fftpoint6 = 0
    fftyvalue6 =0
    
    for j in range(0,k):
        if yf[j] > fftyvalue1:
            fftyvalue1 = yf[j]
            fftpoint1 = j
        elif yf[j] > fftyvalue2:
                fftyvalue2 = yf[j]
                fftpoint2 = j
        elif yf[j] > fftyvalue3:
                fftyvalue3 = yf[j]
                fftpoint3 = j
        elif yf[j] > fftyvalue4:
                fftyvalue4 = yf[j]
                fftpoint4 = j
        elif yf[j] > fftyvalue5:
                fftyvalue5 = yf[j]
                fftpoint5 = j
        elif yf[j] > fftyvalue6:
                fftyvalue6 = yf[j]
                fftpoint6 = j
                
                
#    print("maxfftpoint:", maxfftpoint) #frequency point with maximum
    fftfrequency1 = xf[fftpoint1] #from frequency distribution
    fftfrequency2 = xf[fftpoint2] #from frequency distribution
    fftfrequency3 = xf[fftpoint3] #from frequency distribution
    fftfrequency4 = xf[fftpoint4] #from frequency distribution
    fftfrequency5 = xf[fftpoint5] #from frequency distribution
    fftfrequency6 = xf[fftpoint6] #from frequency distribution

    
    #print("maxfftfrequency: ", maxfftfrequency)
    #input()
    ##m  =  12*log2(fm/440 Hz) + 69
    midiformaxfftfrequency1 = 0 #default if there is no noise in the tone
    midiformaxfftfrequency2 = 0 #default if there is no noise in the tone
    midiformaxfftfrequency3 = 0 #default if there is no noise in the tone
    midiformaxfftfrequency4 = 0 #default if there is no noise in the tone
    midiformaxfftfrequency5 = 0 #default if there is no noise in the tone
    midiformaxfftfrequency6 = 0 #default if there is no noise in the tone
    
    if fftfrequency1 > 10: #minimum must be at leats 10 Hz
        midiformaxfftfrequency1 =  int(round((12*math.log2(fftfrequency1/440) + 69),0)) #m has to be integer
    if fftfrequency2 > 10: #minimum must be at leats 10 Hz
        midiformaxfftfrequency2 =  int(round((12*math.log2(fftfrequency2/440) + 69),0)) #m has to be integer
    if fftfrequency3 > 10: #minimum must be at leats 10 Hz
        midiformaxfftfrequency3 =  int(round((12*math.log2(fftfrequency3/440) + 69),0)) #m has to be integer
    if fftfrequency4 > 10: #minimum must be at leats 10 Hz
        midiformaxfftfrequency4 =  int(round((12*math.log2(fftfrequency4/440) + 69),0)) #m has to be integer
    if fftfrequency5 > 10: #minimum must be at leats 10 Hz
        midiformaxfftfrequency5 =  int(round((12*math.log2(fftfrequency4/440) + 69),0)) #m has to be integer
    if fftfrequency6 > 10: #minimum must be at leats 10 Hz
        midiformaxfftfrequency6 =  int(round((12*math.log2(fftfrequency4/440) + 69),0)) #m has to be integer
        
    #print("maxfftfrequency: ", maxfftfrequency)
    #print("midiformaxfftfrequency: ",midiformaxfftfrequency)
    return(midiformaxfftfrequency1,midiformaxfftfrequency2,midiformaxfftfrequency3,midiformaxfftfrequency4,midiformaxfftfrequency5,midiformaxfftfrequency6)
###########################################MAIN_FUNCTION_FOR_GIVEN_SAMPLE_INTERVAL##################################################

########METHOD 4  = MOVING AREA ANALYSIS BASED ON LOOKING FOR CONSISTENT FREQUENCY OF THE NOTE OVER EXTENDED INTERVALS OF THE SOUND RECORDING
print("METHOD 4: Entering analysis section of the software. This take a bit. No need to press any keys. Thank you.")
points_to_run_analysis = int(samplerate/100) #if 100 - each point will represent 0.01 seconds  = 100 points per second
midisound0 = [] #= this is the output file for midi sounds
midisound1 = []
midisound2 = []
midisound3 = []
midisound4 = []
midisound5 = []


for s0 in range(0,lengthofwavindatapoints,points_to_run_analysis):
    m = returnmidivalueforsamplefromdatapoint(normalized_tone_left,s0,lenghofsampleindatapoints) #normalized _tone_left is left channel of Wav file
    midisound0.append(m)
#minimum for 1-6 maximum points in FFT
for i in range(0,len(midisound0)):
    p = 1000
    for j in range(0,6):
        if midisound0[i][j]<p:
            p = midisound0[i][j]
    midisound1.append(p)
#selection of minimum MIDI in the interval -15 and + 15 codes before gives point (each code is 0.01 seconds = width of selection is 0.30 seconds)
w = 15 #width of minimum MIDI seeking
p=1000
for i in range(0,w):
    p = min(midisound1[0:w])
    midisound2.append(p)
for i in range(w,len(midisound1)-w):
    p = min(midisound1[(i-w):(i+w)])
    midisound2.append(p)
for i in range(len(midisound1)-w,len(midisound1)):
    p = min(midisound1[len(midisound1)-w:len(midisound1)])
    midisound2.append(p)

j=0
midisound3.append([midisound2[0],0.01]) # integrating the midi tones to beat files; each point is 0.01 second
for i in range(1,len(midisound2)):
    t = midisound2[i]
    if t == midisound3[j][0]:
        midisound3[j][1] = midisound3[j][1] + 0.01
    else:
        midisound3.append([midisound2[i],0.01])
        j=j+1

k = len(midisound3)
BPM3Original = 60/(lengthofwavinseconds/k)
midisound3original = []
i=-1
for m in midisound3:
    if tonestype == "instrument":
        cutout = 0.2
    else: #if  tonestype == "human":
        cutout = 0.1
    if m[0]>0 and m[1]>cutout: # longer than 0.2 seconds = 300 BPM for instrument and 0.1s for human voice
        midisound3original.append([findthetonetomidi(m[0]),m[0],m[1],BPM3Original,100]) #100 is volume
        i = i+1
    else:
##        print("midisound3original[i][2]: ",i,midisound3original[i-1][2])
##        input()
##        print("midisound3original[i][2]: ",i,midisound3original[i][2])
##        input()
        if i > 0:
            midisound3original[i][2] = midisound3original[i][2] + m[1] #if note is too short = add that time to the previous note which works
        

print("METHOD 4 = MIDI OUTPUT USING Movings averages approach ORIGINAL. Average BPM is: ",BPM3Original)
for m in midisound3original:
    print(m)
input()


for i in range (0,len(midisound3)):
    durationofthetone = midisound3[i][1]
    m = midisound3[i][0]
    if (durationofthetone <=(1.5*(60/BPM))) and (durationofthetone >=(0.35*(60/BPM))) and (m> 20): #1.5 or 0.35 are important factors for note duration; M = 20 is minimum midi tone
        durationofthetone = 60/BPM # force 1 beat time
        midisound4.append([findthetonetomidi(m),m,durationofthetone,BPM,100])

print("METHOD 4 = MIDI OUTPUT USING Movings averages approach - BPM Forced to BPM: ",BPM)
for i in range(0,len(  midisound4)):
    print(midisound4[i])

input
########END OF METHOD 4  = MOVING AREA ANALYSIS BASED ON LOOKING FOR CONSISTENT FREQUENCY OF THE NOTE OVER EXTENDED INTERVALS OF THE SOUND RECORDING
print("####END OF ANALYTICAL APPROACHES")


##input()
##print("Result of gaps one between tones analysis:", msingle[:][1]) #basdd pon quiet period between sounds
##print("Result of Constant Beat sequence analysis:", msinglebybeat[:][1]) # this is the scale to be played as the resut using Beat division procedure
##print("Result of moving averages  analysis:      ", midisound4[:][1]) # this is the scale to be played as the resut using complex procedure
##print("Result of Spotify Basic Pitch abalysis:   ", midispotify2[:][1]) # this is the scale to be played as the resut using complex procedure
##input()


def midisoundlinsttonmidifile(degrees, BPM,outmidi):
    #degrees  = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number
    #Cello instrument has program number 42
    #https://midiutil.readthedocs.io/en/1.0.1/common.html

##    track   = 0
##    channel = 0
##    time    = len(degrees)# add beats into the composition
##    program = 42 # A Cello
##    MyMIDI.addProgramChange(track, channel, time, program)

#Open Midi file
    MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track  automatically created)
#First define the Track of Midi - we are using only one Track
    track    = 0
    time     = 0   # In beats
    tempo    = BPM  # In BPM
    MyMIDI.addTempo(track,time, tempo)

    channel  = 0
##    duration = 1   # In beats - this 
##    volume   = 127 # 0-127, as per the MIDI standard - Volume can be called Velocity
    #pitch is MIDI channel pitch = Frequency of the Note
    
    for note in degrees:
#       print(note)
#       input()
        pitch = note[1]
        duration = note[2] #in seconds
        duration = duration/(60/BPM) # in Beats
        volume = note[4]
        MyMIDI.addNote(track, channel, pitch, time, duration, volume)
        time = time + duration #before it was 1
    with open(outmidi, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    #https://newt.phys.unsw.edu.au/jw/notes.html
    




msingle1=[]
midisound41=[]
midisound3original1 = []
msinglebybeat1=[]
midispotify21=[]
midispotifyoriginal1=[]


for m in msingle:
    msingle1.append(m[1])
for i in range(0,len(msinglebybeat)):
    msinglebybeat1.append(msinglebybeat[i][1])
for i in range(0,len(midisound4)):
    midisound41.append(midisound4[i][1])
for i in range(0,len(midisound3original)):
    midisound3original1.append(midisound3original[i][1])
for i in range(0,len(midispotify2)):
    midispotify21.append(midispotify2[i][1])
for i in range(0,len(midispotifyoriginal)):
    midispotifyoriginal1.append(midispotifyoriginal[i][1])
    
print("MIDI for Short Sound Analysis: ",BPM,msingle1)
print("MIDI for Beat Analysis:        ",BPM,msinglebybeat1)
print("MIDI for Long Sound Analysis:  ",BPM,midisound41)
print("MIDI for Long Sound Analysis:  ",BPM3Original,midisound3original1)
print("MIDI for Spotify Basic Pitch:  ",BPM,midispotify21)
print("MIDI for Spotify ORIGINAL:  ",BPMoriginal,midispotifyoriginal1)

input()

       
midisoundlinsttonmidifile(msingle, BPM,outmidishortanalysis)
midisoundlinsttonmidifile(msinglebybeat, BPM,outmidibeatanalysis)
midisoundlinsttonmidifile(midisound4, BPM,outmidilonganalysis)
midisoundlinsttonmidifile(midisound3original, BPM3Original,outmidilonganalysisoriginal)
midisoundlinsttonmidifile(midispotify2, BPM,outmidibasicpitch)
midisoundlinsttonmidifile(midispotifyoriginal, BPMoriginal,outmidibasicpitchoriginal)
input()

def printmidifile(outmidishortanalysis, samplerate):
    #https://librosa.org/doc/main/auto_examples/plot_display.html#sphx-glr-auto-examples-plot-display-py
    #librosa.display.specshow(data, *, x_coords=None, y_coords=None, x_axis=None, y_axis=None, sr=22050, hop_length=512, n_fft=None, win_length=None, fmin=None, fmax=None, tuning=0.0, bins_per_octave=12, key='C:maj', Sa=None, mela=None, thaat=None, auto_aspect=True, htk=False, unicode=True, intervals=None, unison=None, ax=None, **kwargs)
    midi_data = pretty_midi.PrettyMIDI(outmidishortanalysis)
    piano_roll = midi_data.get_piano_roll(times=midi_data.get_beats())
    plt.figure()
    librosa.display.specshow(piano_roll, sr=samplerate, x_axis='time', y_axis='cqt_note', fmin=100,fmax = 440)
    #librosa.display.specshow(piano_roll, sr=samplerate, x_axis='time', y_axis='log', fmin=100)
    plt.colorbar()
    plt.show()



playmusicfile(inwav) #origonal file

playmusicfile(outmidishortanalysis)
printmidifile(outmidishortanalysis, samplerate)

playmusicfile(outmidibeatanalysis)
printmidifile(outmidibeatanalysis, samplerate)

playmusicfile(outmidilonganalysis)
printmidifile(outmidilonganalysis, samplerate)

playmusicfile(outmidilonganalysisoriginal)
printmidifile(outmidilonganalysisoriginal, samplerate)

playmusicfile(outmidibasicpitch)
printmidifile(outmidibasicpitch, samplerate)

playmusicfile(outmidibasicpitchoriginal)
printmidifile(outmidibasicpitchoriginal, samplerate)

playmusicfile(inwav)

print("END")




